### predictions

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

from functions.model_data import bestHyperparamaters 
from functions.postproccessing import process_results

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats
%store -r X_test_labeled_df



In [3]:
X_test_labeled_df.head()

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,06_DH1_1,1.43,5.95,2.00,1.37,5.18,806.55,0.250083,635.240000,93.21,27.83,0.34,1.08,443.100000,0.07,2.79,1
1,07_DH1_2,1.47,7.51,1.84,1.93,5.43,807.55,0.250083,639.090000,95.19,27.99,0.24,1.09,442.100000,0.06,2.37,-1
2,08_DH1_3,1.16,4.40,1.93,2.14,2.61,837.73,0.250083,670.480000,93.63,26.43,0.08,1.17,444.280000,0.03,2.46,-1
3,09_DH2_1,1.45,12.48,4.29,1.59,2.87,750.62,0.080000,1515.220000,63.80,31.98,0.25,1.19,369.135675,0.06,18.47,1
4,10_DH2_2,1.99,14.40,4.44,0.81,5.92,796.21,0.090000,710.920579,60.96,44.95,0.51,1.22,369.135675,0.12,19.98,1


In [4]:
train_data_formodel.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.630000,943.71,464944.18,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,0
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,43.735431,1077.11,465010.94,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,0
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.700000,620.21,465295.41,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,0
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.420000,1143.19,402596.61,3735.154885,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,0
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.520000,547.22,465027.11,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,0


### configurations

* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [5]:
save_predictions = True

In [6]:
from config_and_dependencies.config import *

### counts of instances in all classes before oversampling

In [7]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
10     30
7      30
6      30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y and the variables identified as best by the 2 feature_selection notebook are used as features

In [8]:
y = np.array(train_data_formodel[target])
X = np.array(np.array(train_data_formodel[best_feats]))

### the dimensions of the class and features are checked

In [9]:
print(X.shape)
print(y.shape)

(696, 15)
(696,)


### grid search is utilised to identify optimum hyperparamaters

In [ ]:
my_optimiser = bestHyperparamaters(X, y)
my_optimiser.get_best_model()

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
best_model = my_optimiser.best_model

### Model is built for predicting source of artefacts 

In [ ]:
best_model.fit(X, y)

# sanity check to see model correctly predicts geological samples

In [35]:
train_check = best_model.predict(X)
sum(train_check == y)/len(y)

1.0

In [ ]:
train_check == y

In [ ]:
train_check

In [ ]:
X.shape

# When I check the encodings below with that in notebook 1, it is to be correct

In [ ]:
train_check

In [ ]:
print(best_model)

In [ ]:
import pickle


In [ ]:
filename = 'output_datasets/models/rfc_model.sav'
pickle.dump(best_model, open(filename, 'wb'))


In [ ]:
my_dict = {}
numbers = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]
sites = ['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB']

In [ ]:
for i in numbers:
    my_dict[i] = sites[i]

In [ ]:
my_dict

In [ ]:
print(train_data_formodel[target].unique())
print(uniques)

In [ ]:
identifiers =  X_test_labeled_df['Analysis']

In [ ]:
X_test_labeled_df.head()

In [ ]:
results = process_results(best_model, X_test_labeled_df, best_feats, uniques, identifiers)

# test

In [ ]:
y_pred = best_model.predict(np.array(X_test_labeled_df[best_feats]))

In [ ]:
X_test_labeled_df

In [ ]:
y_pred

In [ ]:


y_pred = best_model.predict(np.array(data[best_feats]))
y_pred_proba = best_model.predict_proba(np.array(data[best_feats]))

probabilities_df = pd.DataFrame(data = y_pred_proba, columns = uniques)
probabilities_df_final = pd.concat([probabilities_df, identifiers.reset_index(drop = True)], axis = 1)

final_pred_df = pd.concat([pd.Series(y_pred), probabilities_df_final], axis = 1).rename(columns={0:'class_number'})

final_predictions_df = pd.concat([final_pred_df, data['inlierLabel']], axis = 1)

uniques_list = list(uniques)
def get_pred_names(row):
    return(uniques_list[row['class_number']])

final_predictions_df['class_predictions'] = final_predictions_df.apply(get_pred_names, axis = 1)

def outlierAssigner(row):
    if row['inlierLabel'] == -1:
        return('other')
    else:
        return(row['class_predictions'])

final_predictions_df['class_predictions'] = final_predictions_df.swifter.apply(outlierAssigner, axis = 1)

# test

In [32]:
results

,class_number,FH,ER,WW,TC,CS,KQ,AR,SL,FG,WB,PF,WH,SQ,WN,BH,PH,LB,Analysis,inlierLabel,class_predictions
0,15,0.002500,0.055000,0.046250,0.072500,0.020833,0.045833,0.035417,0.015000,0.055833,0.021250,0.050000,0.012917,0.027500,0.028333,0.076250,0.375417,0.059167,06_DH1_1,1,PH
1,15,0.002917,0.057917,0.041250,0.050000,0.016667,0.049167,0.032917,0.014167,0.058333,0.020000,0.065833,0.009167,0.036667,0.027917,0.091250,0.360000,0.065833,07_DH1_2,-1,other
2,15,0.004167,0.056667,0.063333,0.102917,0.014583,0.047083,0.036667,0.010000,0.042500,0.022500,0.029583,0.010833,0.040417,0.031667,0.103750,0.337917,0.045417,08_DH1_3,-1,other
3,2,0.075417,0.112917,0.298333,0.017083,0.001667,0.029167,0.011250,0.003750,0.089167,0.000417,0.105417,0.001667,0.018333,0.018333,0.074167,0.122917,0.020000,09_DH2_1,1,WW
4,15,0.126667,0.087917,0.132083,0.017917,0.010417,0.017917,0.025000,0.025833,0.044167,0.005833,0.177083,0.000833,0.020417,0.031250,0.040417,0.216667,0.019583,10_DH2_2,1,PH
5,15,0.118750,0.102083,0.149167,0.020833,0.008333,0.014583,0.016250,0.018750,0.039583,0.005417,0.151250,0.001250,0.015417,0.039583,0.062083,0.215000,0.021667,11_DH2_3,1,PH
6,3,0.004167,0.096250,0.067500,0.381250,0.046667,0.023333,0.021667,0.028333,0.002917,0.020833,0.005000,0.075833,0.013333,0.018750,0.009167,0.137500,0.047500,12_DH3_1,-1,other
7,15,0.010417,0.163750,0.119583,0.133000,0.047917,0.007083,0.017500,0.045833,0.009167,0.022917,0.047917,0.030000,0.018333,0.026667,0.032115,0.202500,0.065301,13_DH3_2,-1,other
8,3,0.001667,0.116667,0.067083,0.216667,0.041250,0.088056,0.050486,0.011250,0.009583,0.023056,0.012917,0.084167,0.023750,0.029583,0.015833,0.138750,0.069236,14_DH3_3,1,TC
9,15,0.002917,0.032500,0.026667,0.080833,0.044167,0.009167,0.002500,0.045000,0.005417,0.106667,0.006250,0.019167,0.018750,0.009583,0.087083,0.441667,0.061667,15_DH4_1,1,PH


### I think if there is a problem here it's most likely to be in the encodings of the class, perhaps they are not correct?

In [ ]:
results['class_number'].value_counts()

In [ ]:

#filter for each class prediction, for each class prediction average the probabilities then plot as bar plot, check dists, then filter for outlier or not outlier and bar chart the average probabilities again

### predictions are stored as a variable into memory

In [ ]:
%store results

### predictions are outputted as csv file

In [33]:
if save_predictions:
    results.to_csv('output_datasets/predictions.csv')


In [34]:
results.class_predictions.value_counts()

PH       156
other    151
PF        20
TC        13
WW         7
FG         5
BH         4
LB         2
WB         2
ER         1
WH         1
AR         1
Name: class_predictions, dtype: int64